In [1]:
import requests
import time
import re
from html import unescape

def download(method, url, 
             param=None, data=None,
             timeout=3, maxretries=3):
    try:
        resp = requests.request(method, url, 
                         params=param, data=data, 
                         headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"})

        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if  500 <= e.response.status_code <600 and maxretries>0:
            print(maxretries)
            time.sleep(timeout)
            resp = download(method, url, param, data, timeout, maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [2]:
url = "https://eungdapso.seoul.go.kr/Shr/Shr01/Shr01_lis.jsp"

게시물에서 id받아와서 text파일에 저장

In [ ]:
%%writefile eungdapso.txt



In [251]:
p=re.compile('''\('(\d{14})'\)" title=''')
result = []
with open('C:\\Users\\poco2\\eungdapso.txt' ,'w') as file :
    for i in range(1187):
        html = download('post',url, data = {'page' : i})
        result.extend(p.findall(html.text))
    
    for _ in result:
        file.write(_+'\n')    

text 파일에서 id불러와서 list에넣기

In [5]:
id_list = []

with open('C:\\Users\\poco2\\eungdapso.txt' ,'r') as file :
    id_list.extend(file.read().split('\n'))

In [6]:
id_list = id_list[:-1]

In [7]:
len(id_list)

8792

In [76]:
cont_url = "https://eungdapso.seoul.go.kr/Shr/Shr01/Shr01_vie.jsp"

In [97]:
with open('C:\\Users\\poco2\\eungdapso_content.txt' ,'w') as file :
    for id in id_list[7992:]:
        html = download('post',cont_url, data = {'RCEPT_NO' : id})
        title = re.findall('제목<\/th>\s+<td>(.+)?<\/td>', html.text)[0]
        question = re.findall('상담내용<\/p>\s*(.+?)\s*<\/div>', html.text)[0]
        answer = re.sub('(\\n)|(\\r)|(\\t)','',re.findall('답변내용<\/p>\s*<p>((.?)+\s*(.?)+\s+)<\/p>',html.text)[0][0])

        file.write(title+'*'+question+'*'+answer+'\n')

In [31]:
%%writefile eungdapso_content.txt



Writing eungdapso_content.txt


In [100]:
temp = []
with open('C:\\Users\\poco2\\eungdapso_content.txt' ,'r') as file :
    temp.extend(file.readlines())

In [106]:
(temp[-2].split('*'))

['아현고가도로 철거 관련 문의 ',
 '아현고가도로 철거 관련하여 고가도로에 사용되었던 표지판 등의 기물을 서울시 역사 박물관에 전시할 예정이라고 들었어요. <br/>전시회에 가보려고 하는데 전시 예정일 좀 알려주세요.',
 ' OOO 님 안녕하세요? <br/>서울시에서 추진하고 있는 아현고가도로 철거 사업에 관심을 가져주셔서 감사드리며, 문의하신 아현고가 관련 전시일정에 대하여 말씀드리겠습니다. <br/><br/>2월 중 철거와 함께 현장 유물채취와 자료수집 및 조사 절차 등을 거쳐 5월중에는 전시실을 운영할 계획이나, 정확한 일정은 서울역사박물관 전시일정등 제반사항을 고려하여 결정해야할 사항으로, 일정이 확정되면 서울시역사박물관에서 시민여러분에게 홈페이지 등을 통해 알려드릴 예정입니다.<br/><br/> 또 다른 문의사항 있으시면 언제든지 말씀해 주시기를 부탁드리며, 앞으로도 시에서 추진하고 있는 사업에 많은 관심 가져주시고 좋은 의견 부탁드립니다. <br/><br/>OOO 님 가정에 항상 건강과 큰 기쁨이 가득하시길 기원드립니다. 감사합니다.<br/><br/>[검토부서]  [담당자]  (☎ )  \n']